In [2]:
import pandas as pd


import matplotlib.pyplot as plt
from sklearn.decomposition import NMF 
from sklearn.preprocessing import Normalizer ,MaxAbsScaler
from sklearn.pipeline import make_pipeline

In [3]:
df = pd.read_csv('../data/final_per_90_and_pAdj.csv')

In [4]:


missing = [df.isnull().sum(),100*df.isnull().mean()]

m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head(10)



,count,%
G/SoT,185.0,26.241135
Guaranteed Compensation,181.0,25.673759
name,181.0,25.673759
Unnamed: 0_y,181.0,25.673759
First Name,181.0,25.673759
Last Name,181.0,25.673759
Club,181.0,25.673759
Base Salary,181.0,25.673759
G/Sh,116.0,16.453901
npxG/Sh,116.0,16.453901


In [5]:
# ## dont need club for NMF 

df.drop(columns=['Unnamed: 0_y', 'First Name', 'Last Name', 'name', '_merge','Club','Guaranteed Compensation'],inplace=True)

In [6]:
df.shape

(705, 81)

In [50]:
df.columns

Index(['fb_id', 'player_name', 'Squad_from_op', 'Opposing_Poss',
       'Opposing_Touches', 'Nation', 'Position', 'Nineties', 'xA', 'Key_pass',
       'Comp_prog', 'Total_Cmp', 'Total_Att', 'Total_Cmp_pct',
       'Total_Tot_Dist', 'Total_Prg_Dist', 'Short_Cmp', 'Short_Att',
       'Short_Cmp_pct', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp_pct',
       'Long_Cmp', 'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
       'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
       'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
       'dribble_Succ_pct', 'num_dribble_past', 'dribble_megs', 'carries',
       'carries_dist', 'carries_prog_dist', 'carries_prog', 'carries_att_3rd',
       'carries_att_box', 'miss_control', 'dispossed', 'recep_targ',
       'recep_succ', 'recept_pct', 'Fouls_drawn', 'Offsides', 'Crosses',
       'PK_won', 'Aerial_win_pct', 'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90',
       'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK', 'PK', 'PKatt', 'xG', 'npxG'

In [7]:
df['Base Salary'].sample(5)

109    275000.0
189         NaN
6           NaN
402     66724.0
635     81375.0
Name: Base Salary, dtype: float64

In [8]:
df[['G/SoT','SoT%','npxG/Sh','Dist']].describe()

,G/SoT,SoT%,npxG/Sh,Dist
count,520.000000,589.000000,589.000000,589.000000
mean,0.027278,30.385059,0.013745,2.395677
std,0.078353,19.195458,0.029169,3.507724
min,0.000000,0.000000,0.000398,0.214706
25%,0.000000,18.200000,0.003141,0.730924
50%,0.012031,30.800000,0.005556,1.148649
75%,0.025696,40.000000,0.011765,2.261905
max,1.000000,100.000000,0.400000,26.727273


In [8]:
cols = ['G/Sh','G/SoT','SoT%','npxG/Sh','Dist','Base Salary']

In [9]:

df[['G/Sh','G/SoT','SoT%','npxG/Sh','Dist','Base Salary']].describe()

,G/Sh,G/SoT,SoT%,npxG/Sh,Dist,Base Salary
count,589.000000,520.000000,589.000000,589.000000,589.000000,5.240000e+02
mean,0.009940,0.027278,30.385059,0.013745,2.395677,3.811160e+05
std,0.039429,0.078353,19.195458,0.029169,3.507724,4.843645e+05
min,0.000000,0.000000,0.000000,0.000398,0.214706,6.354700e+04
25%,0.000000,0.000000,18.200000,0.003141,0.730924,9.000000e+04
50%,0.002867,0.012031,30.800000,0.005556,1.148649,2.075000e+05
75%,0.007207,0.025696,40.000000,0.011765,2.261905,4.500000e+05
max,0.500000,1.000000,100.000000,0.400000,26.727273,4.500000e+06


In [10]:
for col in cols:
    print(df[col].mean())
    df[col] = df[col].fillna(df[col].mean())

missing = [df.isnull().sum(),100*df.isnull().mean()]
m = pd.DataFrame(missing)
m = m.T
m.columns=['count', '%']
m = m.sort_values(by='count',ascending=False)
m.head()

0.009939746728555306
0.027277585423982696
30.38505942275043
0.013745106122472362
2.395677172663176
381115.9694656489


,count,%
fb_id,0.0,0.0
carries_att_3rd,0.0,0.0
G/Sh,0.0,0.0
SoT/90,0.0,0.0
Sh/90,0.0,0.0


In [11]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 705 entries, 0 to 704
Data columns (total 81 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   fb_id                       705 non-null    object 
 1   player_name                 705 non-null    object 
 2   Squad_from_op               705 non-null    object 
 3   Opposing_Poss               705 non-null    float64
 4   Opposing_Touches            705 non-null    float64
 5   Nation                      705 non-null    object 
 6   Position                    705 non-null    object 
 7   Nineties                    705 non-null    float64
 8   xA                          705 non-null    float64
 9   Key_pass                    705 non-null    float64
 10  Comp_prog                   705 non-null    float64
 11  Total_Cmp                   705 non-null    float64
 12  Total_Att                   705 non-null    float64
 13  Total_Cmp_pct               705 non

In [13]:
defs = df.loc[df['Position']=='DF']

mini = defs[['player_name', 'Squad_from_op', 'Opposing_Poss',
       'Opposing_Touches', 'Nation', 'Position', 'Nineties', 'xA', 'Key_pass',
       'Comp_prog', 'Total_Cmp', 'Total_Att', 'Total_Cmp_pct',
       'Total_Tot_Dist', 'Total_Prg_Dist', 'Short_Cmp', 'Short_Att',
       'Short_Cmp_pct', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp_pct',
       'Long_Cmp', 'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
       'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
       'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
       'dribble_Succ_pct']]
mini.shape

(227, 34)

In [41]:
mini_df = df[['player_name','Position', 'Nineties', 'xA', 'Key_pass',
       'Comp_prog', 'Total_Cmp', 'Total_Att', 'Total_Cmp_pct',
       'Total_Tot_Dist', 'Total_Prg_Dist',
              'Short_Cmp', 'Short_Att',
       'Short_Cmp_pct', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp_pct',
       'Long_Cmp', 'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
       'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
       'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
       'dribble_Succ_pct']]

mini_df.shape

(705, 30)

In [32]:
mini_df = df[['player_name','Position', 'Nineties', 'xA', 'Key_pass',
       'Comp_prog', 'Total_Cmp', 'Total_Att', 'Total_Cmp_pct',
       'Total_Tot_Dist', 'Total_Prg_Dist',
              'Short_Cmp', 'Short_Att',
       'Short_Cmp_pct', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp_pct',
       'Long_Cmp', 'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
       'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
       'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
       'dribble_Succ_pct',
              'num_dribble_past', 'dribble_megs', 'carries',
       'carries_dist', 'carries_prog_dist', 'carries_prog', 'carries_att_3rd',
       'carries_att_box', 'miss_control', 'dispossed', 'recep_targ',
       'recep_succ', 'recept_pct', 'Fouls_drawn', 'Offsides', 'Crosses',
       'PK_won', 'Aerial_win_pct', 
              'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90',
       'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK',
              'PK', 'PKatt', 'xG', 'npxG',
       'npxG/Sh', 
              
              # cant use negatives
              # 'G-xG',
              # 'np:G-xG',
              
              'Base Salary', 
              'pAdj_Yellow_cards',
       'pAdj_Red_cards', 'pAdj_Second_yellow', 'pAdj_Fouls',
       'pAdj_def_interceptions', 'pAdj_from_mish_tackles_won',
       'pAdj_PK_conceded', 'pAdj_OG', 'pAdj_Recoveries',
       'pAdj_Aerial_Duels_lost'
             ]]

mini_df.shape

(705, 74)

In [33]:
# Create a MaxAbsScaler: scaler
scaler = MaxAbsScaler()

# Create an NMF model: nmf
nmf = NMF(5,max_iter=1000,init='nndsvda')

# Create a Normalizer: normalizer
normalizer = Normalizer()

# Create a pipeline: pipeline
pipeline = make_pipeline(scaler,nmf,normalizer)

# Apply fit_transform to playesr: norm_features
norm_features = pipeline.fit_transform(mini_df.select_dtypes(float))

In [34]:
normed_df = pd.DataFrame(norm_features,index=mini_df.player_name.values)
normed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 705 entries, Carles Gil to Kamron Habibullah
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       705 non-null    float64
 1   1       705 non-null    float64
 2   2       705 non-null    float64
 3   3       705 non-null    float64
 4   4       705 non-null    float64
dtypes: float64(5)
memory usage: 33.0+ KB


In [35]:
normed_df[:15]

,0,1,2,3,4
Carles Gil,0.075549,0.020884,0.125625,0.432183,0.889546
Julian Gressel,0.246671,0.084765,0.323295,0.685445,0.598009
Emanuel Reynoso,0.032036,0.093817,0.000000,0.333145,0.937649
Albert Rusnák,0.157205,0.300049,0.383995,0.648706,0.563014
Maximiliano Moralez,0.258402,0.285628,0.000000,0.581197,0.716837
Luciano Acosta,0.092497,0.203486,0.000000,0.428638,0.875390
Lucas Zelarayán,0.000000,0.235707,0.116278,0.415811,0.870645
Djordje Mihailovic,0.096870,0.284737,0.164157,0.533324,0.773407
Brooks Lennon,0.307630,0.116314,0.472667,0.598545,0.556924
Álvaro Medrán,0.301139,0.012017,0.098961,0.495387,0.808684


In [36]:
tt = normed_df.loc['Tommy Thompson']
sims = normed_df.dot(tt)

sims.nlargest(10)

Tommy Thompson      1.000000
Jack McGlynn        0.994293
Emmanuel Más        0.985904
Tanner Tessmann     0.985420
Joseph Rosales      0.984367
Justin Che          0.981871
Sami Guediri        0.980008
Luciano Abecasis    0.978638
Matheus Rossetto    0.975559
D.J. Taylor         0.974317
dtype: float64

In [37]:
# using all players
tb = normed_df.loc['Tanner Beason']
sims = normed_df.dot(tb)

sims.nlargest(10)

Tanner Beason           1.000000
Henry Kessler           0.999311
Alexander Callens       0.998718
Antonio Carlos          0.998478
Yeimar Gómez Andrade    0.997037
Lalas Abubakar          0.996875
Jhohan Romaña           0.996375
Julio Cascante          0.996346
Sean Nealis             0.996002
Maxime Chanot           0.995972
dtype: float64

In [92]:
defs.loc[defs['Squad_from_op']=='San Jose']

,fb_id,player_name,Squad_from_op,Opposing_Poss,Opposing_Touches,Nation,Position,Nineties,xA,Key_pass,...,pAdj_Yellow_cards,pAdj_Red_cards,pAdj_Second_yellow,pAdj_Fouls,pAdj_def_interceptions,pAdj_from_mish_tackles_won,pAdj_PK_conceded,pAdj_OG,pAdj_Recoveries,pAdj_Aerial_Duels_lost
184,da0bb55a,Paul Marie,San Jose,47.4,548.264706,fr FRA,DF,11.1,0.090090,1.531532,...,5.258195,0.000000,0.0,3.286372,3.450691,2.793416,0.0,0.000000,15.610267,2.957735
226,9b3ab988,Marcos López,San Jose,47.4,548.264706,pe PER,DF,15.7,0.076433,0.828025,...,3.485229,0.000000,0.0,3.020532,5.924889,1.394092,0.0,0.000000,11.849778,2.555835
257,6c510f2d,Luciano Abecasis,San Jose,47.4,548.264706,ar ARG,DF,13.7,0.116788,0.802920,...,3.727753,0.000000,0.0,4.792826,4.127156,3.594619,0.0,0.000000,16.242354,2.662681
269,5f6c69d8,Tommy Thompson,San Jose,47.4,548.264706,us USA,DF,8.3,0.132530,1.204819,...,4.834530,0.000000,0.0,1.318508,5.493785,2.856768,0.0,0.000000,14.503591,1.758011
415,e97787d9,Oswaldo Alanís,San Jose,47.4,548.264706,mx MEX,DF,22.7,0.008811,0.176211,...,0.241049,0.000000,0.0,1.205244,4.499579,1.526643,0.0,0.000000,13.338038,3.294335
455,d4438745,Nathan Cardoso,San Jose,47.4,548.264706,br BRA,DF,19.2,0.020833,0.156250,...,0.189993,0.094997,0.0,4.084858,8.359709,6.174785,0.0,0.000000,15.294467,6.364778
548,3f1b018e,Tanner Beason,San Jose,47.4,548.264706,us USA,DF,21.4,0.009346,0.046729,...,0.340923,0.000000,0.0,1.022768,3.409227,0.937537,0.0,0.085231,12.614140,6.051378


In [117]:
#I need to look at what features are in each category to figure out what to set p too, other hyperparamters 
#need to be tuned 

## def features only below 

In [46]:
mini_df = df[[
    'player_name',
    # 'Position', 
    'Nineties', 
    # 'xA', 'Key_pass',
    #    'Comp_prog', 'Total_Cmp', 'Total_Att', 'Total_Cmp_pct',
    #    'Total_Tot_Dist', 'Total_Prg_Dist',
    #           'Short_Cmp', 'Short_Att',
    #    'Short_Cmp_pct', 'Medium_Cmp', 'Medium_Att', 'Medium_Cmp_pct',
    #    'Long_Cmp', 'Long_Att', 'Long_Cmp_pct', 'touches', 'touches_def_box',
    #    'touches_def_3rd', 'touches_mid_3rd', 'touches_att_3rd',
    #    'touches_att_box', 'touch_live', 'dribble_Succ', 'dribble_Att',
    #    'dribble_Succ_pct',
    #           'num_dribble_past', 'dribble_megs', 'carries',
    #    'carries_dist', 'carries_prog_dist', 'carries_prog', 'carries_att_3rd',
    #    'carries_att_box', 'miss_control', 'dispossed', 'recep_targ',
    #    'recep_succ', 'recept_pct', 'Fouls_drawn', 'Offsides', 'Crosses',
    #    'PK_won', 'Aerial_win_pct', 
    #           'Gls', 'Sh', 'SoT', 'SoT%', 'Sh/90',
    #    'SoT/90', 'G/Sh', 'G/SoT', 'Dist', 'FK',
    #           'PK', 'PKatt', 'xG', 'npxG',
    #    'npxG/Sh', 
              
              # cant use negatives
              # 'G-xG',
              # 'np:G-xG',
              
              'Base Salary', 
              'pAdj_Yellow_cards',
       'pAdj_Red_cards', 'pAdj_Second_yellow', 'pAdj_Fouls',
       'pAdj_def_interceptions', 'pAdj_from_mish_tackles_won',
       'pAdj_PK_conceded', 'pAdj_OG', 'pAdj_Recoveries',
       'pAdj_Aerial_Duels_lost'
             ]]

mini_df.shape

(705, 13)

In [52]:


# Create a MaxAbsScaler: scaler
scaler = MaxAbsScaler()

# Create an NMF model: nmf
nmf = NMF(3,max_iter=1000,init='nndsvda')

# Create a Normalizer: normalizer
normalizer = Normalizer()

# Create a pipeline: pipeline
pipeline = make_pipeline(scaler,nmf,normalizer)

# Apply fit_transform to playesr: norm_features
norm_features = pipeline.fit_transform(mini_df.select_dtypes(float))

In [53]:
normed_df = pd.DataFrame(norm_features,index=mini_df.player_name.values)
normed_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 705 entries, Carles Gil to Kamron Habibullah
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   0       705 non-null    float64
 1   1       705 non-null    float64
 2   2       705 non-null    float64
dtypes: float64(3)
memory usage: 22.0+ KB


In [54]:
tt = normed_df.loc['Tommy Thompson']
sims = normed_df.dot(tt)

sims.nlargest(10)

Tommy Thompson      1.000000
Boniek García       0.999544
Emmanuel Más        0.998708
Frankie Amaya       0.997823
Kamohelo Mokotjo    0.997517
Federico Navarro    0.997034
José Bizama         0.996846
Amro Tarek          0.996454
Wyatt Omsberg       0.996128
Drew Moor           0.993837
dtype: float64

In [55]:
# using all players
tb = normed_df.loc['Tanner Beason']
sims = normed_df.dot(tb)

sims.nlargest(10)

Tanner Beason                1.000000
Antonio Carlos               0.999995
Matt Besler                  0.999979
José Antonio Martínez Gil    0.999954
Leandro González Pirez       0.999874
Larrys Mabiala               0.999865
Yeimar Gómez Andrade         0.999771
Sean Davis                   0.999747
Bill Tuiloma                 0.999668
Oswaldo Alanís               0.999594
dtype: float64

In [56]:
tb = normed_df.loc['Sean Nealis']
sims = normed_df.dot(tb)

sims.nlargest(10)

Sean Nealis           1.000000
Dayne St. Clair       0.999954
Nick Hagglund         0.999581
Jorge Figal           0.999486
Rónald Matarrita      0.999455
Gustavo Vallecilla    0.999424
Joel Waterman         0.999371
John Tolkin           0.999302
Marcelo Silva         0.999276
Tom Edwards           0.999275
dtype: float64

### chaing p give totally different solutions, need to feature engineer and select to more detail to choose which ones are offensive and defenvie or whole , look at what p is to see if joined features make sense or not. 

In [ ]:
def display_topics(model, feature_names, num_top_words,\ topic_names=None):
'''Given an NMF model, feature_names, and number of top words, print topic number and its top feature names, up to specified number of top words.'''
    # iterate through topics in topic-term matrix, 'H' aka
    # model.components_
    for ix, topic in enumerate(model.components_):
        #print topic, topic number, and top words
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i] \
             for i in topic.argsort()[:-num_top_words - 1:-1]]))

In [ ]:
nmf_model = NMF(8)
doc_topic = nmf_model.fit_transform(data_dtm_noun)
display_topics(nmf_model, tv_noun.get_feature_names(), 5)